In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/ING3/NLP/Projet_NLP_chatbot/Projet_VerFinal/dataset'

/content/drive/.shortcut-targets-by-id/1c-DUi14_W785-FFX1R46bJ01W69NheeB/Projet_NLP_chatbot/Projet_VerFinal/dataset


# Imports

## Librairies


In [ ]:
import pandas as pd
import spacy
import re
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from tqdm import tqdm
tqdm.pandas()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Dataset

In [ ]:
data_file = 'wiki_movie_plots_deduped.csv'
dataset = pd.read_csv(data_file)

In [ ]:
dataset.keys()

Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot'],
      dtype='object')

In [ ]:
dataset.head(10)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
5,1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab..."
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...
8,1905,The Little Train Robbery,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Train...,The opening scene shows the interior of the ro...
9,1905,The Night Before Christmas,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Night_Before...,Scenes are introduced using lines of the poem....


# Topic modelling

## Imports

In [ ]:
!python -m spacy download en_core_web_md

2023-12-08 15:34:20.569128: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-08 15:34:20.569195: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-08 15:34:20.569244: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-08 15:34:21.989516: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 27.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
nlp = spacy.load("en_core_web_md", disable=["parser", "ner", "tok2vec"])
# Charger le modèle spaCy
nlp = spacy.load("en_core_web_sm")

stop_words = set(stopwords.words("english"))

## Fonction de préprocessing

Tokenizer (preprocessing niveau mot)

In [ ]:
# suppression des caractères spéciaux
def remove_special_characters(text):
    # Utilise une expression régulière pour supprimer les caractères spéciaux
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

# mettre le texte en minuscule
def lowercase_text(text):
    return text.lower()

# suppression des stop words
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

#lemmatisation
def lemmatize_text(text):
    # Analysez le texte avec spaCy
    doc = nlp(text)

    # Obtenez les lemmes de chaque token
    lemmatized_tokens = [token.lemma_ for token in doc]

    # Rejoignez les lemmes pour former le texte lemmatisé final
    lemmatized_text = " ".join(lemmatized_tokens)

    return lemmatized_text

# tokenisation
def tokenize_text(text):
    # Analysez le texte avec spaCy
    doc = nlp(text)

    # Obtenez les tokens
    tokens = [token.text for token in doc]

    return tokens

In [ ]:
# tokenisation avec tout le traintement de texte au préalable
def tokenize_final_text(text):
  if isinstance(text, str):
    no_special_char = remove_special_characters(text)
    lower_case = lowercase_text(no_special_char)
    no_stop_word = remove_stopwords(lower_case)
    lemmatize = lemmatize_text(no_stop_word)
    final_tokens = tokenize_text(lemmatize)
    return final_tokens
  else:
    return []

## Exemple de tokenization de description

In [ ]:
description = "Hello, blyad,$$, I would like to see a science-fiction from 1985 movie like Star Wars, what would you recommend me ?"
description_tokenized = tokenize_final_text(description)
print(description_tokenized)

['hello', 'blyad', 'would', 'like', 'see', 'sciencefiction', '1985', 'movie', 'like', 'star', 'war', 'would', 'recommend']


# Topic modelling sur le dataset

In [ ]:
dataset['Plot_tokenized'] = dataset['Plot'].progress_apply(tokenize_final_text)

100%|██████████| 34886/34886 [38:10<00:00, 15.23it/s]


In [ ]:
dataset.to_csv('wiki_movie_plots_tokenized.csv')